### Содержание

1. [Получаем контекст последовательности с помощью pyfaidx](#Получаем-контекст-последовательности-с-помощью-pyfaidx)
2. [Автоматизируем получение контекста (прямо из таблицы с данными)](#Автоматизируем-получение-контекста-(прямо-из-таблицы-с-данными))
3. [Пересечение интервалов интересующих позиций](#Пересечение-интервалов-интересующих-позиций)

### Получаем контекст последовательности с помощью pyfaidx

In [105]:
# TODO
# Настроить обработку случаев, когда целевая позиция <= 5

In [1]:
from pyfaidx import Fasta

Загружаем нашу фасту с транскриптами, на этом этапе создается индекс fai.  

Дополнительно кастомизируем ключи, т.к. в файле траснкриптов ключи выглядят так:  
`ENST00000353265.8|ENSG00000178184.16|OTTHUMG00000132922.4|OTTHUMT00000256435.3|PARD6G-201|PARD6G|3836|protein_coding|`  
А нам нужна только их первая часть.

In [77]:
transcript_fasta = Fasta('data_dir/gencode.v45.transcripts.fa.gz', key_function = lambda x: x.split('.')[0])

Интересующий транскрипт и позиция:

In [99]:
transcript_id = 'ENST00000353265'
position_of_interest = 70

Утилита работает как словарь, тогда по примеру из документации:

In [100]:
transcript_fasta[transcript_id][position_of_interest - 6 : position_of_interest + 5].complement
# 64 : 75

>ENST00000353265:65-75 (complement)
GCCCGGCCCCG

Обратите внимание, это **комплементарная** последовательность!

In [80]:
sequence_of_interest = 'GCCCGGCCCCG'

In [81]:
for i, nucleotide in enumerate(sequence_of_interest, 65):
    print(f"{i} - {nucleotide}")

65 - G
66 - C
67 - C
68 - C
69 - G
70 - G
71 - C
72 - C
73 - C
74 - C
75 - G


Проверила глазами соответствия в таблице, всё сходится.

### Автоматизируем получение контекста (прямо из таблицы с данными)

Вот так можно автоматизировать вытягивание контекста, работая напрямую с таблицей tsv:

In [90]:
import pandas as pd

from os.path import join

In [91]:
DATA_PATH = "data_dir"
df = pd.read_table(join(DATA_PATH, "only_for_canonical.tsv"))

In [92]:
transcript_fasta = Fasta('data_dir/gencode.v45.transcripts.fa.gz', key_function = lambda x: x.split('.')[0])

В таблице есть пустые поля в столбце `cDNA_position`, и они при итерации отмечаются как `nan` (т.е. `<class 'float'>`).  
  
Если значение не пустое, оно имеет тип `str`, так что для работы `pyfaidx` приходится приводить его к `int`.

In [106]:
for index, row in df.iterrows():
    transcript_id = row['Canonical_transcript']
    position_of_interest = row['cDNA_position']
    #print(transcript_id, ':', position_of_interest, type(position_of_interest))

    if isinstance(position_of_interest, str) and '-' not in position_of_interest:
        position_of_interest = int(position_of_interest)

        if position_of_interest > 5:  # иначе получается 5 - 6 в левой границе
            sequence_of_interest = transcript_fasta[transcript_id][position_of_interest - 6 : position_of_interest + 5].complement

            print("Transcript ID:", transcript_id)
            print("Position of interest:", position_of_interest)
            print("Context of interest:", sequence_of_interest[:5], sequence_of_interest[5], sequence_of_interest[6:])
            print('\n')

    else:
        print("Skip row with cDNA_position value:", position_of_interest)
        print('\n')

### Пересечение интервалов интересующих позиций

In [3]:
import pandas as pd
from os.path import join

In [29]:
DATA_PATH = "data_dir"
data = pd.read_table(join(DATA_PATH, "deduplicated.tsv"))
data.head()

,Chr,Position,rsID,Ref,Alt,Consequence,Canonical,Gene_symbol,Domains,LoF,LoF_flag,LoF_filter
0,chr18,46050,rs744323,C,T,"['ENST00000308911: downstream_gene_variant', '...","ENST00000308911: YES, ENST00000572530: YES, EN...","TUBB8B, LOC105371950",NaN,NaN,NaN,NaN
1,chr18,47239,.,TTTA,T,"['ENST00000308911: 3_prime_UTR_variant', 'ENST...","ENST00000308911: YES, ENST00000572530: YES, EN...","TUBB8B, LOC105371950",NaN,NaN,NaN,NaN
2,chr18,47242,rs893233952,A,G,"['ENST00000308911: 3_prime_UTR_variant', 'ENST...","ENST00000308911: YES, ENST00000572530: YES, EN...","TUBB8B, LOC105371950",NaN,NaN,NaN,NaN
3,chr18,47243,.,T,C,"['ENST00000308911: 3_prime_UTR_variant', 'ENST...","ENST00000308911: YES, ENST00000572530: YES, EN...","TUBB8B, LOC105371950",NaN,NaN,NaN,NaN
4,chr18,47245,.,A,T,"['ENST00000308911: 3_prime_UTR_variant', 'ENST...","ENST00000308911: YES, ENST00000572530: YES, EN...","TUBB8B, LOC105371950",NaN,NaN,NaN,NaN


In [7]:
data.shape

(339, 12)

Из распарсенной таблицы отобрали только строки, содержащие непустые значения в одном из лоф-полей (LoF/LoF_flag/LoF_filter)  

In [30]:
filtered_data = data[(~data['LoF'].isna()) | (~data['LoF_flag'].isna()) | (~data['LoF_filter'].isna())]

In [21]:
filtered_data.shape

(10, 12)

Перевели в vcf

In [32]:
vcf_df = filtered_data[['Chr', 'Position', 'rsID', 'Ref', 'Alt']] 
vcf_df.loc[:, 'QUAL'] = "."
vcf_df.loc[:, 'FILTER'] = "."
vcf_df.loc[:, 'INFO'] = "."

header = """##fileformat=VCFv4.2
#CHROM POS ID REF ALT QUAL FILTER INFO
"""

output_VCF = "data_dir/myfile.vcf"
with open(output_VCF, 'w') as vcf:
    vcf.write(header)

vcf_df.to_csv(output_VCF, sep="\t", mode='a', index=False, header=False)

/tmp/ipykernel_4995/4217350662.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vcf_df.loc[:, 'QUAL'] = "."
/tmp/ipykernel_4995/4217350662.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vcf_df.loc[:, 'FILTER'] = "."
/tmp/ipykernel_4995/4217350662.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

Сделала  
! bedtools intersect -wo -a myfile.vcf -b gencode.v45.annotation.gtf.gz > intersected_annotations.gtf

Получился файл intersected_annotations.gtf